# 멀티 에이전트 애플리케이션에서 다중 턴 대화를 추가하는 방법 (함수형 API)

!!! info "필수 조건"
    이 가이드는 다음에 대한 익숙함을 가정합니다:

    - [멀티 에이전트 시스템](../../concepts/multi_agent)
    - [Human-in-the-loop](../../concepts/human_in_the_loop)
    - [함수형 API](../../concepts/functional_api)
    - [Command](../../concepts/low_level/#command)
    - [LangGraph 용어집](../../concepts/low_level/)


이 how-to 가이드에서는 최종 사용자가 하나 이상의 에이전트와 *다중 턴 대화*를 할 수 있는 애플리케이션을 구축합니다. 사용자 입력을 수집하기 위해 [`interrupt`](../../reference/types/#langgraph.types.interrupt)를 사용하고 **활성** 에이전트로 다시 라우팅하는 노드를 만들 것입니다.

에이전트는 에이전트 단계를 실행하고 다음 작업을 결정하는 워크플로우의 작업으로 구현됩니다:

1. **사용자 입력을 기다림**하여 대화를 계속하거나,
2. [**핸드오프**](../../concepts/multi_agent/#handoffs)를 통해 **다른 에이전트로 라우팅**(또는 루프와 같이 자기 자신에게 다시 라우팅).

```python
from langgraph.func import entrypoint, task
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langgraph.types import interrupt


# 다른 에이전트로 핸드오프할 의도를 나타내는 도구를 정의합니다
# 참고: 다른 에이전트로 이동하기 위해 Command(goto) 구문을 사용하지 않습니다:
# 아래의 `workflow()`가 핸드오프를 명시적으로 처리합니다
@tool(return_direct=True)
def transfer_to_hotel_advisor():
    """호텔 어드바이저 에이전트에게 도움을 요청합니다."""
    return "Successfully transferred to hotel advisor"


# 에이전트를 정의합니다
travel_advisor_tools = [transfer_to_hotel_advisor, ...]
travel_advisor = create_react_agent(model, travel_advisor_tools)


# 에이전트를 호출하는 작업을 정의합니다
@task
def call_travel_advisor(messages):
    response = travel_advisor.invoke({"messages": messages})
    return response["messages"]


# 멀티 에이전트 네트워크 워크플로우를 정의합니다
@entrypoint(checkpointer)
def workflow(messages):
    call_active_agent = call_travel_advisor
    while True:
        agent_messages = call_active_agent(messages).result()
        ai_msg = get_last_ai_msg(agent_messages)
        if not ai_msg.tool_calls:
            user_input = interrupt(value="Ready for user input.")
            messages = messages + [{"role": "user", "content": user_input}]
            continue

        messages = messages + agent_messages
        call_active_agent = get_next_agent(messages)
    return entrypoint.final(value=agent_messages[-1], save=messages)
```

## 설정

먼저 필요한 패키지를 설치하겠습니다

In [1]:
# %%capture --no-stderr
# %pip install -U langgraph langchain-anthropic

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

ANTHROPIC_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 문제를 빠르게 발견하고 LangGraph 프로젝트의 성능을 개선하세요. LangSmith를 사용하면 LangGraph로 구축한 LLM 앱을 디버그, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있습니다 — 시작하는 방법에 대해 자세히 알아보려면 <a href="https://docs.smith.langchain.com">여기</a>를 참조하세요. 
    </p>
</div>

이 예제에서는 서로 통신할 수 있는 여행 어시스턴트 에이전트 팀을 구축합니다.

2개의 에이전트를 만들 것입니다:

* `travel_advisor`: 여행 목적지 추천을 도울 수 있습니다. `hotel_advisor`에게 도움을 요청할 수 있습니다.
* `hotel_advisor`: 호텔 추천을 도울 수 있습니다. `travel_advisor`에게 도움을 요청할 수 있습니다.

이것은 완전히 연결된 네트워크입니다 - 모든 에이전트는 다른 에이전트와 통신할 수 있습니다.

In [ ]:
import random
from typing_extensions import Literal
from langchain_core.tools import tool


@tool
def get_travel_recommendations():
    """여행 목적지에 대한 추천을 받습니다"""
    return random.choice(["aruba", "turks and caicos"])


@tool
def get_hotel_recommendations(location: Literal["aruba", "turks and caicos"]):
    """주어진 목적지에 대한 호텔 추천을 받습니다."""
    return {
        "aruba": [
            "The Ritz-Carlton, Aruba (Palm Beach)"
            "Bucuti & Tara Beach Resort (Eagle Beach)"
        ],
        "turks and caicos": ["Grace Bay Club", "COMO Parrot Cay"],
    }[location]


@tool(return_direct=True)
def transfer_to_hotel_advisor():
    """호텔 어드바이저 에이전트에게 도움을 요청합니다."""
    return "Successfully transferred to hotel advisor"


@tool(return_direct=True)
def transfer_to_travel_advisor():
    """여행 어드바이저 에이전트에게 도움을 요청합니다."""
    return "Successfully transferred to travel advisor"

!!! note "전송 도구"

    전송 도구에서 `@tool(return_direct=True)`를 사용하고 있다는 것을 알아차렸을 것입니다. 이는 개별 에이전트(예: `travel_advisor`)가 이러한 도구가 호출되면 ReAct 루프를 조기에 종료할 수 있도록 하기 위한 것입니다. 이것이 원하는 동작입니다. 에이전트가 이 도구를 호출할 때를 감지하고 _즉시_ 다른 에이전트에게 제어를 넘기고 싶기 때문입니다.
    
    **참고**: 이것은 사전 빌드된 [`create_react_agent`][langgraph.prebuilt.chat_agent_executor.create_react_agent]와 함께 작동하도록 되어 있습니다 -- 사용자 정의 에이전트를 구축하는 경우 `return_direct`로 표시된 도구에 대한 조기 종료 처리 로직을 수동으로 추가해야 합니다.

이제 사전 빌드된 [`create_react_agent`][langgraph.prebuilt.chat_agent_executor.create_react_agent]와 멀티 에이전트 워크플로우를 사용하여 에이전트를 만들어 봅시다. 각 에이전트로부터 최종 응답을 받은 후 매번 [`interrupt`][langgraph.types.interrupt]를 호출할 것입니다.

In [ ]:
import uuid

from langchain_core.messages import AIMessage
from langchain_anthropic import ChatAnthropic
from langgraph.prebuilt import create_react_agent
from langgraph.graph import add_messages
from langgraph.func import entrypoint, task
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import interrupt, Command

model = ChatAnthropic(model="claude-3-5-sonnet-latest")

# 여행 어드바이저 ReAct 에이전트를 정의합니다
travel_advisor_tools = [
    get_travel_recommendations,
    transfer_to_hotel_advisor,
]
travel_advisor = create_react_agent(
    model,
    travel_advisor_tools,
    prompt=(
        "You are a general travel expert that can recommend travel destinations (e.g. countries, cities, etc). "
        "If you need hotel recommendations, ask 'hotel_advisor' for help. "
        "You MUST include human-readable response before transferring to another agent."
    ),
)


@task
def call_travel_advisor(messages):
    # 에이전트에 대한 입력/출력 변경 등 추가 로직을 추가할 수도 있습니다.
    # 참고: 상태의 전체 메시지 기록으로 ReAct 에이전트를 호출하고 있습니다
    response = travel_advisor.invoke({"messages": messages})
    return response["messages"]


# 호텔 어드바이저 ReAct 에이전트를 정의합니다
hotel_advisor_tools = [get_hotel_recommendations, transfer_to_travel_advisor]
hotel_advisor = create_react_agent(
    model,
    hotel_advisor_tools,
    prompt=(
        "You are a hotel expert that can provide hotel recommendations for a given destination. "
        "If you need help picking travel destinations, ask 'travel_advisor' for help."
        "You MUST include human-readable response before transferring to another agent."
    ),
)


@task
def call_hotel_advisor(messages):
    response = hotel_advisor.invoke({"messages": messages})
    return response["messages"]


checkpointer = InMemorySaver()


def string_to_uuid(input_string):
    return str(uuid.uuid5(uuid.NAMESPACE_URL, input_string))


@entrypoint(checkpointer=checkpointer)
def multi_turn_graph(messages, previous):
    previous = previous or []
    messages = add_messages(previous, messages)
    call_active_agent = call_travel_advisor
    while True:
        agent_messages = call_active_agent(messages).result()
        messages = add_messages(messages, agent_messages)
        # 마지막 AI 메시지를 찾습니다
        # 핸드오프 도구 중 하나가 호출되면 에이전트가 반환한 마지막 메시지는
        # "return_direct=True"로 설정했기 때문에 ToolMessage가 됩니다.
        # 이는 마지막 AIMessage가 도구 호출을 가질 것임을 의미합니다.
        # 그렇지 않으면 마지막으로 반환된 메시지는 도구 호출이 없는
        # AIMessage가 되며, 이는 새 입력을 받을 준비가 되었음을 의미합니다.
        ai_msg = next(m for m in reversed(agent_messages) if isinstance(m, AIMessage))
        if not ai_msg.tool_calls:
            user_input = interrupt(value="Ready for user input.")
            # 사용자 입력을 사람 메시지로 추가합니다
            # 참고: 내용을 기반으로 사람 메시지에 대한 고유 ID를 생성합니다
            # 이는 중요합니다. 후속 호출에서 이전 사용자 입력(interrupt) 값이
            # 다시 조회되고 여기에 다시 추가하려고 시도하기 때문입니다
            # `add_messages`는 ID를 기반으로 메시지를 중복 제거하여 올바른 메시지 기록을 보장합니다
            human_message = {
                "role": "user",
                "content": user_input,
                "id": string_to_uuid(user_input),
            }
            messages = add_messages(messages, [human_message])
            continue

        tool_call = ai_msg.tool_calls[-1]
        if tool_call["name"] == "transfer_to_hotel_advisor":
            call_active_agent = call_hotel_advisor
        elif tool_call["name"] == "transfer_to_travel_advisor":
            call_active_agent = call_travel_advisor
        else:
            raise ValueError(f"Expected transfer tool, got '{tool_call['name']}'")

    return entrypoint.final(value=agent_messages[-1], save=messages)

## 다중 턴 대화 테스트

이 애플리케이션으로 다중 턴 대화를 테스트해 봅시다.

In [ ]:
thread_config = {"configurable": {"thread_id": uuid.uuid4()}}

inputs = [
    # 1차 대화,
    {
        "role": "user",
        "content": "i wanna go somewhere warm in the caribbean",
        "id": str(uuid.uuid4()),
    },
    # `interrupt`를 사용하고 있으므로 Command 프리미티브를 사용하여 재개해야 합니다.
    # 2차 대화,
    Command(
        resume="could you recommend a nice hotel in one of the areas and tell me which area it is."
    ),
    # 3차 대화,
    Command(
        resume="i like the first one. could you recommend something to do near the hotel?"
    ),
]

for idx, user_input in enumerate(inputs):
    print()
    print(f"--- Conversation Turn {idx + 1} ---")
    print()
    print(f"User: {user_input}")
    print()
    for update in multi_turn_graph.stream(
        user_input,
        config=thread_config,
        stream_mode="updates",
    ):
        for node_id, value in update.items():
            if isinstance(value, list) and value:
                last_message = value[-1]
                if isinstance(last_message, dict) or last_message.type != "ai":
                    continue
                print(f"{node_id}: {last_message.content}")